In [ ]:
from read_data import get_training, get_test, get_Doc2Vec, get_data
from processing import combine_with_vec, exclude_non_numeric
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
import lightgbm as lgb
import pickle

In [ ]:
X,y = get_training()
train_name_vec50, train_ingr_vec50, train_steps_vec50 = get_Doc2Vec(data="train", num_features=50)
train_name_vec100, train_ingr_vec100, train_steps_vec100 = get_Doc2Vec(data="train", num_features=100)

### Doc2Vec preparation

In [ ]:
# Combine Doc2Vec with 50 features and non-numeric train data
temp_X = X.copy()
temp_X = exclude_non_numeric(temp_X)
temp_train_name_vec50 = train_name_vec50.copy()
temp_train_ingr_vec50 = train_ingr_vec50.copy()
temp_train_steps_vec50 = train_steps_vec50.copy()
X_combined_50: pd.DataFrame = combine_with_vec(temp_X, temp_train_name_vec50, temp_train_ingr_vec50, temp_train_steps_vec50)

## Decision Tree pipelines

In [ ]:
# You need to subtract 1 from y because it needs to be 0, 1, 2 instead of 1, 2, 3 for LightGBM
X_train_50_LGBM, X_test_50_LGBM, y_train_50_LGBM, y_test_50_LGBM = train_test_split(X_combined_50, y - 1, test_size=0.2, random_state=30027)

In [ ]:
LGBM_pipeline = make_pipeline(StandardScaler(), lgb.LGBMClassifier())

In [ ]:
%%time
partial_LGBM_model = LGBM_pipeline.fit(X_train_50_LGBM, y_train_50_LGBM)

In [ ]:
partial_LGBM_model.score(X_test_50_LGBM, y_test_50_LGBM)
# -> 0.73

In [ ]:
%%time
LGBM_model = LGBM_pipeline.fit(X_combined_50, y - 1)

In [ ]:
pickle.dump(LGBM_model, open("models/LightGBM-50Doc2Vec.sav", "wb"))